## Open Street Map Project

In this project we will work with the dataset of the city of New York to check for the quality of the data. 
The inputs in this dataset were made by people, which means that most likely there are errors to correct.
We will analyze a sample of the dataset because the original one is too large (2+ GB).

In the upcoming cells, I will explain the code I'm using and what the results mean.
I will provide the explanations in this notebook, so that the entire process is more understandable.
Besides that, I will provide a separate report with more detailed explanation and the necessary attachments.

In [27]:
#Here we take a sample of the original dataset, which is a XML OSM file.

import xml.etree.cElementTree as ET 

OSM_FILE = "newyork.osm"  
SAMPLE_FILE = "sample_ny.osm"

k = 40

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element).decode('utf-8'))

    output.write('</osm>')

In [1]:
import xml.etree.cElementTree as ET 
newyork = 'sample_ny.osm'

def get_root(fname):

    tree = ET.parse(fname)
    return tree.getroot()


get_root(newyork)

<Element 'osm' at 0x03CDC0F0>

In [2]:
#Since we are dealing with OSM, we are working with nodes, ways and relations. With this code, we want to have an overview of the
#structure of the dataset.

import pprint
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
ny_tags = count_tags(newyork)
pprint.pprint(ny_tags)

{'member': 2036,
 'nd': 364027,
 'node': 288441,
 'osm': 1,
 'relation': 249,
 'tag': 243056,
 'way': 45216}


Here below we explore what types of keys we have. We classify them in four categories:
1) lower: for tags that contain only lowercase letters and are valid,
2) lower_colon: for otherwise valid tags with a colon in their names,
3) problemchars: for tags with problematic characters
4) other: for other tags that do not fall into the other three categories.

In [3]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

ny_keys = process_map(newyork)
pprint.pprint(ny_keys)

{'lower': 94919, 'lower_colon': 144683, 'other': 2830, 'problemchars': 624}


In [4]:
#We will audit some criteria of the dataset and clean them.

from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

mapping = {'Ave'  : 'Avenue',
           'AVE.'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane', 
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'st'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square",
           'Tpke' : 'Turnpike'
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

First we pay attention to the street formatting. With this code, we are gathering the types of streets.
In the output below, we notice that there are different formats referring to the same thing.
For instance, "AVE." and "AVENUE" refer to avenue. Same for "st" and "ST"; they both refer to street.
We are getting a set of the different types, so that we can convert them to a standardized form. I.e. we will clean the data in this regard.

In [5]:
ny_street_types = audit(newyork)
pprint.pprint(dict(ny_street_types))

{'1': {'Cushing St # 1'},
 '109': {'Route 109'},
 '1603': {'STREET 1603'},
 '25A': {'Route 25A'},
 '35': {'NJ 35'},
 '4B': {'Union Avenue 4B'},
 '500': {'Main St., Suite 500'},
 '506': {'Broadway #506'},
 '59': {'E Route 59'},
 'A': {'Avenue A'},
 'AVE.': {'NEW ENGLAND AVE.'},
 'AVENUE': {'COLLEGE AVENUE', 'COMMERCIAL AVENUE'},
 'Alley': {'Mac Dougal Alley', 'Cortlandt Alley'},
 'Americas': {'Avenue Of The Americas', 'Avenue of the Americas'},
 'Ave': {'Millard Ave'},
 'B': {'Avenue B'},
 'Bayside': {'Bayside'},
 'Blvd': {'Orchard Beach Blvd', 'Bell Blvd'},
 'Bowery': {'Bowery'},
 'Brandon': {'Brandon'},
 'Broadway': {'West Broadway', 'East Broadway', 'Broadway'},
 'C': {'Avenue C'},
 'CIRCLE': {'BERRUE CIRCLE'},
 'Cir': {'Alister Cir',
         'Carto Cir',
         'Chestnut Cir',
         'Dix Cir',
         'Hendel Cir',
         'Legends Cir',
         'Mystic Cir',
         'Sterling Cir',
         'White Spruce Cir',
         'Winnecomac Cir'},
 'Circle': {'64th Circle',
       

Now we will standardize the names of the streets. We will convert the different versions of the same format into one format.
For instance, instead of Lewis St we will have now Lewis Street.

In [6]:
def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

for street_type, ways in ny_street_types.items():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print (name, "=>", better_name)

Bloomfield St => Bloomfield Street
Lewis St => Lewis Street
Paterson St => Paterson Street
Jackson St => Jackson Street
Grand St => Grand Street
Garden St => Garden Street
7th St => 7th Street
Adams St => Adams Street
1st St => 1st Street
Jefferson St => Jefferson Street
W 35th st => W 35th Street
Eastern Parkway Extension => Eastern Parkway Extension
Flatbush Avenue Extension => Flatbush Avenue Extension
West Broadway => West Broadway
East Broadway => East Broadway
Broadway => Broadway
Jericho Turnpike => Jericho Turnpike
Hempstead Turnpike => Hempstead Turnpike
Union Turnpike => Union Turnpike
Orchard Beach Blvd => Orchard Beach Boulevard
Bell Blvd => Bell Boulevard
Selvin Loop => Selvin Loop
Larrison Loop => Larrison Loop
Millennium Loop => Millennium Loop
Lorraine Loop => Lorraine Loop
Simmons Loop => Simmons Loop
Gwenn Loop => Gwenn Loop
Alan Loop => Alan Loop
Savo Loop => Savo Loop
Chess Loop => Chess Loop
Pembrook Loop => Pembrook Loop
Devon Loop => Devon Loop
El Camino Loop => 

Cadman Plaza West => Cadman Plaza West
Drumgoole Road West => Drumgoole Road West
Biel Road West => Biel Road West
Clearview Expressway Service Road West => Clearview Expressway Service Road West
Willow Road West => Willow Road West
Henry Hudson Parkway West => Henry Hudson Parkway West
Berry Avenue West => Berry Avenue West
Prospect Park West => Prospect Park West
Whitestone Expressway Service Road West => Whitestone Expressway Service Road West
Central Park West => Central Park West
Belmar Drive West => Belmar Drive West
Canal Street West => Canal Street West
Grand Central Parkway Service Road West => Grand Central Parkway Service Road West
Van Cortlandt Avenue West => Van Cortlandt Avenue West
Park Terrace West => Park Terrace West
Van Wyck Expressway Service Road West => Van Wyck Expressway Service Road West
Avenue R => Avenue R
NEW ENGLAND AVE. => NEW ENGLAND Avenue
Avenue M => Avenue M
Avenue T => Avenue T
Avenue O => Avenue O
Avenue D => Avenue D
Avenue Of The Americas => Avenue

Here we will audit the zipcodes in the dataset of New York.
We expect the zipcodes (i.e. postal codes) to have 5 digits. 
There are though formats of (5 digits - 4 digits).
In the code, we set the condition to actually have digits in the zipcode.
And we want to see which codes have more than 5 digits.

In [7]:
def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    if len(zipcode) != 5:
        invalid_zipcodes[twoDigits].add(zipcode)
        
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

ny_zipcode = audit_zip(newyork)

In the output below we notice that there are very few problems.
There is just one minor inconsistency, where the code starts with "NY".
So we strip just the zipcode in digits from those instances.
There are cases with the format of (5 digits - 4 digits). But these are valid formats.
Also, there is one zipcode that has six digits (115422). It is probably a typo error. We don't know which digits are incorrect, so we will leave it like that.

In [8]:
pprint.pprint(dict(ny_zipcode))

{'08': {'08854-8037',
        '08854-8039',
        '08854-8041',
        '08854-8067',
        '08901-1340',
        '08901-8502',
        '08901-8504',
        '08901-8520',
        '08901-8524',
        '08901-8541'},
 '11': {'11374-2756', '115422'},
 'NY': {'NY 10002', 'NY 10533'}}


We strip just the digits from the zipcodes that start with "NY".

In [9]:
def update_zipname(zipcode):
    testNum = re.findall('[a-zA-Z]*', zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip()
    if testNum == "NY":
        convertedZipcode = (re.findall(r'\d+', zipcode))
        if convertedZipcode:
            if convertedZipcode.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]

for street_type, ways in ny_zipcode.items():
    for name in ways:
        better_name = update_zipname(name)
        print (name, "=>", better_name)

NY 10002 => 10002
NY 10533 => 10533
11374-2756 => None
115422 => None
08854-8039 => None
08901-8502 => None
08854-8041 => None
08854-8037 => None
08901-8524 => None
08901-1340 => None
08901-8504 => None
08901-8520 => None
08901-8541 => None
08854-8067 => None


With the same logic, we will audit the housenumbers and check for inconsistency.

In [10]:
housenr_types_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def is_housenr(elem):
    return (elem.attrib['k'] == "addr:housenumber")

def audit_housenr_type(housenr_types, housenr):
    m = housenr_types_re.search(housenr)
    if m:
        housenr_type = m.group()
        housenr_types[housenr_type].add(housenr)

def audit_housenr_types(osmfile):
    osm_file = open(osmfile, "r")
    housenr_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_housenr(tag):
                    audit_housenr_type(housenr_types, tag.attrib['v'])
    return housenr_types

In [11]:
ny_housenr_types = audit_housenr_types(newyork)

In New York, housenumbers are usually formed of digits, digits and fractions, digits and letters.
In the output below we can see such formats.
However, we also notice formats like (digits + "REAR(A/B)", and even "Park Street".
These are clearly inconsistent formats of house number, which we must clean.

In [12]:
pprint.pprint(dict(ny_housenr_types))

{'1': {'1'},
 '1-24': {'1-24'},
 '1-42': {'1-42'},
 '1/2': {'101 1/2',
         '105 1/2',
         '113 1/2',
         '115-21 1/2',
         '122-16 1/2',
         '130 1/2',
         '134 1/2',
         '141 1/2',
         '141-01 1/2',
         '148 1/2',
         '159 1/2',
         '170 1/2',
         '172 1/2',
         '187 1/2',
         '2291 1/2',
         '237 1/2',
         '259 1/2',
         '263 1/2',
         '274 1/2',
         '281 1/2',
         '283 1/2',
         '307 1/2',
         '32 1/2',
         '435 1/2',
         '52 1/2',
         '525 1/2',
         '60-38 1/2',
         '707 1/2',
         '735 1/2',
         '74-06 1/2',
         '76 1/2',
         '84-22 1/2',
         '93 1/2',
         '94 1/2'},
 '1/3': {'380 1/3', '568 1/3'},
 '1/4': {'3184 1/4'},
 '10': {'10'},
 '10-01': {'10-01'},
 '10-02': {'10-02'},
 '10-06': {'10-06'},
 '10-16': {'10-16'},
 '10-17': {'10-17'},
 '10-18': {'10-18'},
 '10-20': {'10-20'},
 '10-27': {'10-27'},
 '10-33': {'10-33'},

 '120A': {'120A'},
 '121': {'121'},
 '121-01': {'121-01'},
 '121-02': {'121-02'},
 '121-03': {'121-03'},
 '121-04': {'121-04'},
 '121-07': {'121-07'},
 '121-08': {'121-08'},
 '121-09': {'121-09'},
 '121-10': {'121-10'},
 '121-15': {'121-15'},
 '121-16': {'121-16'},
 '121-17': {'121-17'},
 '121-20': {'121-20'},
 '121-27': {'121-27'},
 '121-31': {'121-31'},
 '121-34': {'121-34'},
 '121-36': {'121-36'},
 '121-42': {'121-42'},
 '121-45': {'121-45'},
 '121-47': {'121-47'},
 '121-63': {'121-63'},
 '1210': {'1210'},
 '1211': {'1211'},
 '1212': {'1212'},
 '1213': {'1213'},
 '1214': {'1214'},
 '1214A': {'1214A'},
 '1215': {'1215'},
 '1216': {'1216'},
 '1217': {'1217'},
 '1218': {'1218'},
 '1219': {'1219'},
 '121D': {'121D'},
 '122': {'122'},
 '122-02': {'122-02'},
 '122-05': {'122-05'},
 '122-06': {'122-06'},
 '122-07': {'122-07'},
 '122-11': {'122-11'},
 '122-12': {'122-12'},
 '122-13': {'122-13'},
 '122-14': {'122-14'},
 '122-19': {'122-19'},
 '122-20': {'122-20'},
 '122-21': {'122-21'},
 '12

 '1462': {'1462'},
 '1463': {'1463'},
 '1464': {'1464'},
 '1465': {'1465'},
 '1466': {'1466'},
 '1467': {'1467'},
 '1468': {'1468'},
 '1469': {'1469'},
 '147': {'147'},
 '147-01': {'147-01'},
 '147-04': {'147-04'},
 '147-05': {'147-05'},
 '147-06': {'147-06'},
 '147-07': {'147-07'},
 '147-09': {'147-09'},
 '147-11': {'147-11'},
 '147-12': {'147-12'},
 '147-13': {'147-13'},
 '147-14': {'147-14'},
 '147-15': {'147-15'},
 '147-16': {'147-16'},
 '147-17': {'147-17'},
 '147-18': {'147-18'},
 '147-19': {'147-19'},
 '147-20': {'147-20'},
 '147-21': {'147-21'},
 '147-22': {'147-22'},
 '147-23': {'147-23'},
 '147-25': {'147-25'},
 '147-26': {'147-26'},
 '147-28': {'147-28'},
 '147-29': {'147-29'},
 '147-32': {'147-32'},
 '147-34': {'147-34'},
 '147-35': {'147-35'},
 '147-37': {'147-37'},
 '147-38': {'147-38'},
 '147-41': {'147-41'},
 '147-45': {'147-45'},
 '147-48': {'147-48'},
 '147-49': {'147-49'},
 '147-51': {'147-51'},
 '147-55': {'147-55'},
 '147-57': {'147-57'},
 '147-59': {'147-59'},
 '1

 '1679': {'1679'},
 '167A': {'167A'},
 '167B': {'167B'},
 '168': {'168'},
 '168-01': {'168-01'},
 '168-02': {'168-02'},
 '168-04': {'168-04'},
 '168-08': {'168-08'},
 '168-10': {'168-10'},
 '168-14': {'168-14'},
 '168-19': {'168-19'},
 '168-20': {'168-20'},
 '168-21': {'168-21'},
 '168-35': {'168-35'},
 '168-39': {'168-39'},
 '168-40': {'168-40'},
 '168-43': {'168-43'},
 '168-48': {'168-48'},
 '168-60': {'168-60'},
 '1680': {'1680'},
 '1681': {'1681'},
 '1682': {'1682'},
 '1683': {'1683'},
 '1684': {'1684'},
 '1685': {'1685'},
 '1686': {'1686'},
 '1689': {'1689'},
 '168A': {'168A'},
 '168B': {'168B'},
 '169': {'169'},
 '169-01': {'169-01'},
 '169-02': {'169-02'},
 '169-04': {'169-04'},
 '169-05': {'169-05'},
 '169-12': {'169-12'},
 '169-14': {'169-14'},
 '169-16': {'169-16'},
 '169-19': {'169-19'},
 '169-20': {'169-20'},
 '169-24': {'169-24'},
 '169-26': {'169-26'},
 '169-43': {'169-43'},
 '1690': {'1690'},
 '1691': {'1691'},
 '1692': {'1692'},
 '1693': {'1693'},
 '1694': {'1694'},
 '1

 '209-14': {'209-14'},
 '209-25': {'209-25'},
 '209-31': {'209-31'},
 '209-36': {'209-36'},
 '209-43': {'209-43'},
 '209-48': {'209-48'},
 '209-60': {'209-60'},
 '209-62': {'209-62'},
 '209-70': {'209-70'},
 '209-85': {'209-85'},
 '2092': {'2092'},
 '2093': {'2093'},
 '2094': {'2094'},
 '2095': {'2095'},
 '2096': {'2096'},
 '2098': {'2098'},
 '20A': {'20A'},
 '21': {'21'},
 '21-01': {'21-01'},
 '21-04': {'21-04'},
 '21-09': {'21-09'},
 '21-11': {'21-11'},
 '21-12': {'21-12'},
 '21-13': {'21-13'},
 '21-15': {'21-15'},
 '21-16': {'21-16'},
 '21-17': {'21-17'},
 '21-18': {'21-18'},
 '21-19': {'21-19'},
 '21-20': {'21-20'},
 '21-23': {'21-23'},
 '21-24': {'21-24'},
 '21-25': {'21-25'},
 '21-26': {'21-26'},
 '21-27': {'21-27'},
 '21-28': {'21-28'},
 '21-29': {'21-29'},
 '21-29A': {'21-29A'},
 '21-30': {'21-30'},
 '21-31': {'21-31'},
 '21-33': {'21-33'},
 '21-34': {'21-34'},
 '21-35': {'21-35'},
 '21-36': {'21-36'},
 '21-37': {'21-37'},
 '21-38': {'21-38'},
 '21-42': {'21-42'},
 '21-44': {'2

...

 '3255': {'3255'},
 '3259': {'3259'},
 '325A': {'325A'},
 '325b': {'325b'},
 '326': {'326'},
 '3260': {'3260'},
 '3267': {'3267'},
 '3268': {'3268'},
 '326A': {'326A'},
 '326F': {'326F'},
 '327': {'327'},
 '3270': {'3270'},
 '3272': {'3272'},
 '3277': {'3277'},
 '3278': {'3278'},
 '328': {'328'},
 '3281': {'3281'},
 '3287B': {'3287B'},
 '329': {'329'},
 '3290': {'3290'},
 '3292': {'3292'},
 '3293': {'3293'},
 '3294': {'3294'},
 '32A': {'32A'},
 '33': {'33'},
 '33-01': {'33-01'},
 '33-02': {'33-02'},
 '33-04': {'33-04'},
 '33-08': {'33-08'},
 '33-09': {'33-09'},
 '33-10': {'33-10'},
 '33-11': {'33-11'},
 '33-12': {'33-12'},
 '33-14': {'33-14'},
 '33-15': {'33-15'},
 '33-16': {'33-16'},
 '33-17': {'33-17'},
 '33-18': {'33-18'},
 '33-19': {'33-19'},
 '33-21': {'33-21'},
 '33-22': {'33-22'},
 '33-23': {'33-23'},
 '33-25': {'33-25'},
 '33-27': {'33-27'},
 '33-28': {'33-28'},
 '33-29': {'33-29'},
 '33-32': {'33-32'},
 '33-33': {'33-33'},
 '33-34': {'33-34'},
 '33-35': {'33-35'},
 '33-36': {'

 '45-11': {'45-11'},
 '45-12': {'45-12'},
 '45-14': {'45-14'},
 '45-15': {'45-15'},
 '45-18': {'45-18'},
 '45-20': {'45-20'},
 '45-21': {'45-21'},
 '45-22': {'45-22'},
 '45-23': {'45-23'},
 '45-24': {'45-24'},
 '45-25': {'45-25'},
 '45-29': {'45-29'},
 '45-30': {'45-30'},
 '45-31': {'45-31'},
 '45-34': {'45-34'},
 '45-38': {'45-38'},
 '45-39': {'45-39'},
 '45-40': {'45-40'},
 '45-42': {'45-42'},
 '45-43': {'45-43'},
 '45-44': {'45-44'},
 '45-46': {'45-46'},
 '45-47': {'45-47'},
 '45-53': {'45-53'},
 '45-54': {'45-54'},
 '45-55': {'45-55'},
 '45-57': {'45-57'},
 '45-59': {'45-59'},
 '45-65': {'45-65'},
 '45-68': {'45-68'},
 '45-70': {'45-70'},
 '45-72A': {'45-72A'},
 '45-79': {'45-79'},
 '450': {'450'},
 '4501': {'4501'},
 '4508': {'4508'},
 '4509': {'4509'},
 '450A': {'450A'},
 '451': {'451'},
 '4510': {'4510'},
 '4511': {'4511'},
 '4512': {'4512'},
 '4515': {'4515'},
 '4516': {'4516'},
 '4518': {'4518'},
 '452': {'452'},
 '4521': {'4521'},
 '4523': {'4523'},
 '4526': {'4526'},
 '452A'

 '6146': {'6146'},
 '615': {'615'},
 '616': {'616'},
 '616A': {'616A'},
 '617': {'617'},
 '618': {'618'},
 '619': {'619'},
 '61A': {'61A'},
 '62': {'62'},
 '62-01': {'62-01'},
 '62-02': {'62-02'},
 '62-05': {'62-05'},
 '62-06': {'62-06'},
 '62-07': {'62-07'},
 '62-08': {'62-08'},
 '62-11': {'62-11'},
 '62-12': {'62-12'},
 '62-14': {'62-14'},
 '62-15': {'62-15'},
 '62-22': {'62-22'},
 '62-23': {'62-23'},
 '62-29': {'62-29'},
 '62-30': {'62-30'},
 '62-35': {'62-35'},
 '62-37': {'62-37'},
 '62-40': {'62-40'},
 '62-44': {'62-44'},
 '62-47': {'62-47'},
 '62-48': {'62-48'},
 '62-51': {'62-51'},
 '62-52': {'62-52'},
 '62-58': {'62-58'},
 '62-59': {'62-59'},
 '62-60': {'62-60'},
 '62-66': {'62-66'},
 '62-75': {'62-75'},
 '62-76': {'62-76'},
 '62-82': {'62-82'},
 '62-86': {'62-86'},
 '62-95': {'62-95'},
 '62-97': {'62-97'},
 '620': {'620'},
 '6202': {'6202'},
 '6203': {'6203'},
 '621': {'621'},
 '6216': {'6216'},
 '622': {'622'},
 '6224': {'6224'},
 '623': {'623'},
 '624': {'624'},
 '625': {'62

 '80-47': {'80-47'},
 '80-50': {'80-50'},
 '80-51': {'80-51'},
 '80-52': {'80-52'},
 '80-55': {'80-55'},
 '80-56': {'80-56'},
 '80-58': {'80-58'},
 '80-64': {'80-64'},
 '80-67': {'80-67'},
 '80-68': {'80-68'},
 '80-70': {'80-70'},
 '80-72': {'80-72'},
 '80-79': {'80-79'},
 '80-84': {'80-84'},
 '800': {'800'},
 '8001': {'8001'},
 '8002': {'8002'},
 '8003': {'8003'},
 '801': {'801'},
 '8011': {'8011'},
 '8013': {'8013'},
 '8015': {'8015'},
 '8019': {'8019'},
 '802': {'802'},
 '8022': {'8022'},
 '8023': {'8023'},
 '8025': {'8025'},
 '803': {'803'},
 '804': {'804'},
 '805': {'805'},
 '806': {'806'},
 '807': {'807'},
 '808': {'808'},
 '809': {'809'},
 '81': {'81'},
 '81-04': {'81-04'},
 '81-05': {'81-05'},
 '81-07': {'81-07'},
 '81-08': {'81-08'},
 '81-10': {'81-10'},
 '81-12': {'81-12'},
 '81-14': {'81-14'},
 '81-16': {'81-16'},
 '81-17': {'81-17'},
 '81-18': {'81-18'},
 '81-1A': {'81-1A'},
 '81-20': {'81-20'},
 '81-21': {'81-21'},
 '81-22': {'81-22'},
 '81-25': {'81-25'},
 '81-27': {'81-2

 '97': {'97'},
 '97-02': {'97-02'},
 '97-03': {'97-03'},
 '97-04': {'97-04'},
 '97-06': {'97-06'},
 '97-07': {'97-07'},
 '97-08': {'97-08'},
 '97-09': {'97-09'},
 '97-1': {'97-1'},
 '97-10': {'97-10'},
 '97-11': {'97-11'},
 '97-12': {'97-12'},
 '97-13': {'97-13'},
 '97-14': {'97-14'},
 '97-15': {'97-15'},
 '97-16': {'97-16'},
 '97-17': {'97-17'},
 '97-19': {'97-19'},
 '97-20': {'97-20'},
 '97-21': {'97-21'},
 '97-23': {'97-23'},
 '97-25': {'97-25'},
 '97-26': {'97-26'},
 '97-27': {'97-27'},
 '97-28': {'97-28'},
 '97-29': {'97-29'},
 '97-30': {'97-30'},
 '97-33': {'97-33'},
 '97-35': {'97-35'},
 '97-36': {'97-36'},
 '97-39': {'97-39'},
 '97-40': {'97-40'},
 '97-41': {'97-41'},
 '97-44': {'97-44'},
 '97-49': {'97-49'},
 '97-79': {'97-79'},
 '970': {'970'},
 '9708': {'9708'},
 '971': {'971'},
 '972': {'972'},
 '9720': {'9720'},
 '9724': {'9724'},
 '9727': {'9727'},
 '973': {'973'},
 '9736': {'9736'},
 '974': {'974'},
 '975': {'975'},
 '976': {'976'},
 '976A': {'976A'},
 '977': {'977'},
 '

We define a new audit function to loop through every housenumber in the dataset.

In [13]:
def audit_housenr(osmfile):
    osm_file = open(osmfile, "r")
    housenr = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_housenr(tag):
                    housenr.append(tag.attrib['v'])
    return housenr
    
ny_housenr = audit_housenr(newyork)
pprint.pprint((ny_housenr))

['4',
 '153',
 '332',
 '205',
 '300',
 '1233',
 '500',
 '600',
 '732',
 '457',
 '832',
 '932',
 '1233',
 '832',
 '240-16',
 '150',
 '300',
 '25',
 '108',
 '185',
 '225',
 'Park Street',
 '23-02',
 '386',
 '226',
 '1407',
 '281-295',
 '790',
 '150',
 '1473',
 '797',
 '786',
 '203',
 '8101',
 '23',
 '1373',
 '69',
 '1024',
 '3475',
 '597',
 '2501',
 '55',
 '892',
 '676',
 '380',
 '722',
 '270',
 '2400',
 '82',
 '4615',
 '645',
 '300',
 '2276',
 '2440',
 '51',
 '19',
 '140',
 '45',
 '175',
 '238',
 '216',
 '19',
 '80',
 '409',
 '368',
 '425 ',
 '760',
 '1955',
 '31',
 '196',
 '210',
 '311',
 '197-03',
 '198-35',
 '35',
 '1601',
 '84',
 '70',
 '1701',
 '1030',
 '399',
 '111',
 '3',
 '39',
 '7',
 '1074D',
 '609',
 '2079',
 '416',
 '301',
 '567',
 '268',
 '486',
 '205',
 '281 1/2',
 '89',
 '120',
 '314',
 '175',
 '1384',
 '35',
 '430',
 '1745',
 '129',
 '107',
 '161B',
 '468',
 '85',
 '723',
 '560',
 '108',
 '46-21',
 '228',
 '433',
 '77',
 '557',
 '440',
 '135',
 '198',
 '400',
 '58',
 '107

 '622',
 '2102',
 '2723',
 '921A',
 '969A',
 '854',
 '2925',
 '1196',
 '1347',
 '3524',
 '1238',
 '1871',
 '900',
 '1001',
 '1125',
 '3327',
 '3857',
 '4202',
 '1447',
 '3324',
 '2222',
 '1467',
 '1290',
 '728',
 '645',
 '320',
 '40',
 '80',
 '157',
 '88B',
 '254',
 '108',
 '80',
 '1673',
 '1508',
 '1684',
 '3423',
 '3502',
 '3018',
 '1884',
 '1902',
 '3521',
 '448',
 '509',
 '4122',
 '2102',
 '4024',
 '1144',
 '687',
 '1384',
 '218',
 '1056',
 '1804',
 '5024',
 '1757',
 '1193',
 '5401',
 '580',
 '5802',
 '1988',
 '1554',
 '1670',
 '5912',
 '634',
 '1651',
 '1002',
 '13',
 '9036',
 '10313',
 '1149',
 '8718',
 '906',
 '8015',
 '770',
 '45',
 '1480',
 '8123',
 '1111',
 '1251',
 '8922',
 '1473',
 '9424',
 '2003',
 '901',
 '645',
 '9902',
 '1032',
 '10002',
 '9814',
 '1411',
 '1523',
 '1027',
 '1275',
 '1404',
 '115',
 '304',
 '10001',
 '833',
 '10583',
 '265',
 '690',
 '1388',
 '534',
 '186',
 '9736',
 '1580',
 '1094',
 '1016',
 '3523',
 '1581',
 '584',
 '1054',
 '2131',
 '791',
 '7515',


 '42-20',
 '43-29',
 '210-07',
 '211-02',
 '39-10',
 '220-02',
 '39-27',
 '215-02',
 '38-15',
 '40-01',
 '34-37',
 '207-02',
 '209-60',
 '35-29',
 '213-05',
 '28-02',
 '200-01',
 '34-66',
 '203-20',
 '200-20',
 '203-06',
 '40-22',
 '45-54',
 '56-02',
 '56-07',
 '56-27',
 '56-40',
 '58-14A',
 '48-76',
 '192-08',
 '196-19',
 '40-05',
 '45-21',
 '59-52',
 '157-02',
 '52-25',
 '59-10',
 '54-01',
 '171-15A',
 '166-06',
 '46-63',
 '43-78',
 '192-10A',
 '194-03',
 '195-14A',
 '36-02',
 '189-11',
 '35-19',
 '36-03',
 '163-32',
 '42-18',
 '28-01',
 '35-64',
 '153-20',
 '150-23',
 '161-10',
 '41-10',
 '156-02',
 '45-72A',
 '142-02',
 '149-01',
 '143-02',
 '41-87',
 '140-11',
 '34-01',
 '144-02',
 '38-32',
 '29-01',
 '149-02',
 '150-15',
 '25-02',
 '16-39',
 '149-01',
 '157-49',
 '19-02',
 '25-39',
 '27-10',
 '25-18',
 '29-64',
 '38-23',
 '34-21',
 '58-01',
 '134-29',
 '137-10',
 '36-09',
 '136-67',
 '40-01',
 '41-31',
 '132-02',
 '135-29',
 '39-22',
 '131-37',
 '41-23',
 '60-01',
 '132-15',
 '57

 '370',
 '44',
 '507',
 '560',
 '7',
 '105',
 '151',
 '189',
 '212',
 '23',
 '246',
 '263',
 '283',
 '32',
 '41',
 '67',
 '243',
 '323',
 '12A',
 '1995',
 '224',
 '250',
 '285',
 '317',
 '339',
 '357',
 '381',
 '16',
 '1882',
 '22',
 '253',
 '282',
 '32',
 '362',
 '392',
 '44',
 '47',
 '506',
 '57',
 '65',
 '77',
 '92',
 '12',
 '18',
 '28',
 '36',
 '421',
 '48',
 '508A',
 '538',
 '625',
 '78',
 '158',
 '180',
 '201',
 '229',
 '276',
 '32A',
 '418',
 '54',
 '701',
 '83',
 '9A',
 '11',
 '120',
 '129',
 '149',
 '174',
 '211',
 '30',
 '41',
 '49',
 '59',
 '67',
 '81',
 '93',
 '1618',
 '1651',
 '1677',
 '1696',
 '1720',
 '1737',
 '1764',
 '1796',
 '1855',
 '1091',
 '1191',
 '4912',
 '650',
 '683',
 '722',
 '742',
 '780',
 '1144',
 '1176',
 '1197',
 '1221',
 '1626',
 '4709',
 '5220',
 '1654',
 '4619',
 '1044',
 '1084',
 '1127',
 '5210',
 '825',
 '877',
 '951',
 '107',
 '125',
 '144',
 '18',
 '212',
 '25',
 '295',
 '35',
 '41',
 '49',
 '54',
 '58',
 '61',
 '65',
 '5622',
 '580',
 '5913',
 '62

 '975',
 '1421',
 '1458',
 '1558',
 '1857',
 '1401',
 '1632',
 '1663',
 '1761',
 '2013',
 '1103',
 '186',
 '198',
 '216',
 '223',
 '639',
 '160-13',
 '161-13',
 '161-32',
 '170-21',
 '172-35',
 '83-10',
 '83-58',
 '85-11',
 '191',
 '205',
 '215',
 '230',
 '290 REAR',
 '795',
 '65',
 '71',
 '122',
 '21',
 '2410',
 '2802',
 '2922',
 '833',
 '68',
 '30',
 '50',
 '73',
 '2100',
 '12',
 '154',
 '179',
 '26',
 '398',
 '225',
 '237',
 '271',
 '719',
 '236',
 '297',
 '379',
 '698',
 '173',
 '401',
 '776',
 '19',
 '253',
 '297',
 '324',
 '128',
 '529',
 '111',
 '410',
 '259',
 '425',
 '151',
 '555',
 '229',
 '1145',
 '55',
 '122',
 '150',
 '162',
 '195',
 '262',
 '310',
 '350',
 '453',
 '58',
 '130',
 '177',
 '414',
 '477',
 '62',
 '110',
 '150',
 '62',
 '97',
 '102',
 '132',
 '215',
 '327',
 '50',
 '197',
 '13',
 '23',
 '373',
 '431',
 '65',
 '87',
 '13',
 '286',
 '307',
 '33',
 '349',
 '382',
 '413',
 '100',
 '215',
 '278',
 '373',
 '399',
 '83',
 '115',
 '204',
 '300',
 '327',
 '95',
 '147',

...

 '102-19',
 '47-21',
 '49-12',
 '50-14',
 '50-32',
 '51-10',
 '53-10',
 '99-03',
 '55-16',
 '90-21',
 '90-38',
 '90-59',
 '92-39',
 '53-06',
 '53-21',
 '53-39',
 '54-11',
 '55-09',
 '55-19',
 '55-31',
 '97-07',
 '51-12',
 '54-08',
 '92-14',
 '92-27',
 '92-37',
 '93-22',
 '94-23',
 '45-02',
 '45-14',
 '47-16',
 '47-58',
 '50-02',
 '50-41',
 '52-01',
 '96-07',
 '96-24',
 '97-25',
 '97-79',
 '47-14',
 '94-27',
 '94-43',
 '94-60',
 '91-28',
 '92-20',
 '51-09',
 '87-62',
 '88-18',
 '88-41',
 '90-10',
 '90-27',
 '90-47',
 '91-06',
 '48-16',
 '48-29',
 '50-08',
 '87-11',
 '88-19',
 '89-10',
 '81-22',
 '82-50',
 '83-30',
 '80-25',
 '81-55',
 '83-25',
 '47-03',
 '89-14',
 '45-15',
 '74-11',
 '77-10',
 '80-32',
 '42-18',
 '42-33',
 '42-52',
 '42-70',
 '44-10',
 '76-15',
 '72-46',
 '41-29',
 '41-60',
 '74-17',
 '75-11',
 '41-11',
 '41-52',
 '73-17',
 '45-12',
 '41-18',
 '41-56',
 '40-12',
 '79-08',
 '37-24',
 '37-40',
 '37-58',
 '37-66',
 '40-28',
 '40-52',
 '33-36',
 '32-19',
 '32-37',
 '32-56',

 '150-31',
 '150-39',
 '150-49',
 '150-63',
 '150-76',
 '152-15',
 '153-11',
 '153-26',
 '154-05',
 '117-11',
 '117-29',
 '119-15',
 '119-33',
 '120-39',
 '123-26',
 '123-50',
 '145-41',
 '146-17',
 '147-17',
 '117-30',
 '119-18',
 '119-35',
 '120-03',
 '120-18',
 '120-35',
 '123-16',
 '142-15',
 '142-49',
 '143-09',
 '144-14',
 '145-17',
 '117-11',
 '117-22',
 '117-32',
 '117-43',
 '117-56',
 '119-18',
 '120-25',
 '140-14',
 '142-14',
 '120-15',
 '121-45',
 '122-45',
 '123-52',
 '124-66',
 '129-19',
 '129-34',
 '129-55',
 '130-34',
 '131-10',
 '131-23',
 '131-37',
 '132-08',
 '134-09',
 '135-38',
 '117-39',
 '117-62',
 '119-53',
 '120-22',
 '120-66',
 '125-19',
 '128-06',
 '129-01',
 '130-16',
 '130-30',
 '130-48',
 '129-20',
 '130-06',
 '131-04',
 '131-20',
 '131-43',
 '132-15',
 '133-10',
 '133-18',
 '133-26',
 '133-40',
 '134-44',
 '117-36',
 '117-58',
 '122-32',
 '124-05',
 '126-12',
 '130-11',
 '130-30',
 '133-02',
 '133-16',
 '133-26',
 '133-35',
 '133-44',
 '123-16',
 '126-18',

 '43-66',
 '45-24',
 '45-47',
 '45-70',
 '158-01',
 '159-18',
 '161-08',
 '43-21',
 '43-39',
 '43-56',
 '45-01',
 '45-29',
 '45-57',
 '45-79',
 '46-23',
 '46-41',
 '46-59',
 '190-06',
 '191-06',
 '192-09',
 '194-09',
 '32-10',
 '33-08',
 '33-44',
 '34-18',
 '36-12',
 '36-25',
 '169-14',
 '190-10',
 '35-14',
 '35-23',
 '35-32',
 '35-44',
 '35-58',
 '36-20',
 '165-12',
 '36-18',
 '36-32',
 '154-10',
 '159-07',
 '163-03',
 '41-08',
 '41-20',
 '42-16',
 '42-24',
 '42-32A',
 '42-40',
 '42-48',
 '150-12',
 '150-44',
 '151-23',
 '152-09',
 '154-18',
 '158-11',
 '28-14',
 '29-11',
 '30-15',
 '30-27',
 '32-02',
 '32-16',
 '32-22',
 '32-30',
 '35-18',
 '150-42',
 '152-18',
 '33-11',
 '33-23',
 '33-33',
 '33-47',
 '33-71',
 '35-24',
 '147-29',
 '149-09',
 '149-48',
 '152-28',
 '154-20',
 '39-25',
 '40-15',
 '40-27',
 '41-20',
 '41-46',
 '146-33',
 '148-07',
 '149-18',
 '149-39',
 '156-09',
 '43-02',
 '43-33',
 '43-63',
 '45-31',
 '45-55',
 '46-11',
 '46-41',
 '141-01 1/2',
 '143-15',
 '45-11',
 '

 '120',
 '206',
 '3',
 '28',
 '112',
 '18',
 '42',
 '20',
 '47',
 '92',
 '14',
 '47',
 '2',
 '9',
 '207',
 '1',
 '27',
 '4',
 '29',
 '44',
 '7',
 '69',
 '11',
 '15',
 '25',
 '29',
 '111',
 '24',
 '47',
 '100',
 '22',
 '93',
 '33',
 '45',
 '28',
 '28',
 '235',
 '31',
 '116',
 '13',
 '34',
 '36',
 '2',
 '8',
 '302',
 '39',
 '62',
 '2',
 '12',
 '32',
 '62',
 '202',
 '40',
 '61',
 '1',
 '3',
 '285',
 '143',
 '1',
 '838',
 '2',
 '305',
 '1',
 '200',
 '212',
 '11',
 '21',
 '237',
 '25',
 '5',
 '40',
 '26',
 '23',
 '14',
 '17',
 '65',
 '8',
 '5',
 '61',
 '8',
 '61',
 '82',
 '10',
 '7',
 '6',
 '28',
 '14',
 '3',
 '40',
 '35',
 '26',
 '116',
 '34',
 '10',
 '36',
 '38',
 '46',
 '5',
 '60',
 '37',
 '48',
 '18',
 '36',
 '48',
 '24',
 '37',
 '27',
 '38',
 '3',
 '27',
 '2',
 '5',
 '10',
 '131',
 '366',
 '5',
 '48',
 '239',
 '8',
 '16',
 '50',
 '61',
 '34',
 '4',
 '2',
 '16',
 '16',
 '27',
 '23',
 '3',
 '44',
 '206',
 '208',
 '103',
 '119',
 '238',
 '863',
 '6',
 '415',
 '113',
 '64',
 '18',
 '503',


 '12',
 '56',
 '16',
 '34',
 '7',
 '494',
 '25',
 '55',
 '24',
 '64',
 '37',
 '57',
 '113',
 '140',
 '146',
 '40',
 '700',
 '506',
 '45',
 '313',
 '6',
 '225',
 '166',
 '34',
 '34',
 '44',
 '50',
 '22',
 '57',
 '159',
 '290',
 '161',
 '67',
 '39',
 '106',
 '90',
 '56',
 '85',
 '89',
 '46',
 '98',
 '93',
 '40',
 '974',
 '58',
 '61',
 '159',
 '75',
 '230',
 '621',
 '24',
 '194',
 '701',
 '7',
 '215',
 '32',
 '21',
 '93',
 '147',
 '53',
 '922',
 '946',
 '153',
 '256',
 '950',
 '38',
 '70',
 '740',
 '40',
 '325',
 '210',
 '202',
 '6',
 '48',
 '220',
 '10',
 '235',
 '2',
 '172',
 '11',
 '3',
 '2',
 '34',
 '56',
 '107',
 '168',
 '23',
 '31',
 '58',
 '34',
 '399',
 '65',
 '21',
 '24',
 '119',
 '117',
 '509',
 '142',
 '17',
 '318',
 '416',
 '224',
 '338',
 '188',
 '221',
 '225',
 '192',
 '417',
 '406',
 '275',
 '7',
 '307',
 '195',
 '288',
 '641',
 '233',
 '187',
 '116',
 '224',
 '298',
 '368',
 '805',
 '1364',
 '230',
 '216',
 '165',
 '99',
 '88',
 '192',
 '141',
 '25',
 '968',
 '306',
 '404'

We clean the housenumbers to get rid of the instances where we have added words (e.g. REAR, Park Street etc.).

In [14]:
def cleanHouse(housenumber):
    clean = []
    for item in housenumber.split():
        for ele in item:
            if ele.isdigit():
                clean.append(item + ' ')
                break
            
            else:
                print('none')
    clean = ''.join(clean)
    return clean

for housenumber in ny_housenr:
    print (cleanHouse(housenumber))

4 
153 
332 
205 
300 
1233 
500 
600 
732 
457 
832 
932 
1233 
832 
240-16 
150 
300 
25 
108 
185 
225 
none
none
none
none
none
none
none
none
none
none

23-02 
386 
226 
1407 
281-295 
790 
150 
1473 
797 
786 
203 
8101 
23 
1373 
69 
1024 
3475 
597 
2501 
55 
892 
676 
380 
722 
270 
2400 
82 
4615 
645 
300 
2276 
2440 
51 
19 
140 
45 
175 
238 
216 
19 
80 
409 
368 
425 
760 
1955 
31 
196 
210 
311 
197-03 
198-35 
35 
1601 
84 
70 
1701 
1030 
399 
111 
3 
39 
7 
1074D 
609 
2079 
416 
301 
567 
268 
486 
205 
281 1/2 
89 
120 
314 
175 
1384 
35 
430 
1745 
129 
107 
161B 
468 
85 
723 
560 
108 
46-21 
228 
433 
77 
557 
440 
135 
198 
400 
58 
107 
159 
337 
1809 
722 
141 
113 1/2 
236 
152 
1526 
1223 
539A 
109 
61 
521 
93 
725 
23-037 
30-018 
353 
431 
572 
131 
216B 
506 
1885 
188 
255B 
258B 
270B 
325A 
340B 
365A 
18-040 
30-002 
7401A 
1140 
261 
15 
66A 
114 
45 
1000U 
92 
14 
127 
172 
42 
86B 
528 
352 
80 
672 
64-20 
1992 
169 
217 
69 
338 
253 
201 

294 
529 
574 
177 
665 
836 
235 
209 
737 
1480 
590 
1553 
500 
2600 
1591 
500 
2620 
727 
4532 
17 
108 
4710 
3 
51 
4746 
3800 
170 
88 
189 
3843A 
147 
255 
91 
519B 
45 
4054 
390 
129 
99 
251 
367 
692 
161 
252 
484S 
463 
311 
249 
2491 
2700 
300 
331 
410 
352 
481 
732 
718 
552 
448 
410 
2868 
395 
500 
616A 
727 
577 
574 
745 
2950 
680 
645 
133 
721 
794 
810 
829 
521 
757D 
783B 
806 
3131 
873 
843 
927 
880 
897A 
984 
905 
301 
3340 
1110 
3475 
3566 
1278 
423 
1151 
250 
1273 
260 
1109 
188 
195 
115 
118 
1158 
6A 
85 
1006 
60 
150 
84 
1180 
878 
540 
831 
749 
866 
756 
900 
875 
914 
940 
967A 
1002 
985 
1077 
1093B 
1125 
1083 
1023 
1201 
1231 
1190 
1377 
1020 
1470 
931 
1480 
980 
940 
1580 
683 
1667 
571 
1427 
1462 
3812 
231 
1643 
330 
1704 
1562 
331 
382 
1101 
998 
949 
1810 
1780 
961 
2152 
883 
2088 
1951 
824 
2017 
2012 
660 
708 
2296 
735 1/2 
685 
2395 
607 
681 
2200 
2164 
637 
4170 
539 
652 
2323 
4652 
2502 
2458 
4563A 
54

43-11 
128 
20-37 
183 
11 
77 
161 
60 
20-70 
282 
456 
143 
54 
184 
38 
87 
375 
134 
174 
318 
474 
584 
1816 
1840 
1845 
26 
24 
255 
226 
42 
93 
209 
110 
95 
103 
209 
101 
119 
26-06 
14-56 
12-16 
26-06 
26-40 
27-04 
18-08 
30-18 
14-48 
14-46 
124 
138 
96 
85 
75A 
163 
129 
51 
142 
117A 
147 
198 
36 
66 
7722 
7516 
1949 
1850 
1915 
1941 
1972 
1820 
1962 
1088 
1186 
132 
153 
170 
199 
228 
25 
281 
32 
40 
55 
657 
none
none
none
none
73 
86 
1219 
16 
25 
36 
49 
65 
89 
11 
165 
40 
64 
99 
101 
112 
124 
134 
149 
17 
201 
25 
295 
43 
56 
67 
81 
96 
239 
42 
161 
73 
183 
33 
69 
123 
149 
18 
226 
26 
298 
329 
37 
43 
50 
64 
11 
156 
210 
6 
131 
185 
25 
306 
345 
380 1/3 
65 
89 
100 
118 
148 
165 
195 
215 
26 
35 
5 
7 
9 
10 
18 
26 
300 
362 
400 
488 
54 
732 
110 
15 
241 
355 
409 
445 
488 
578 
84 
120 
173 
210 
240 
30 
379 
5 
59 
642 
709 
79 
126 
168 
200 
236 
28 
42 
524 
549 
572 
592 
618 
650 
702 
11 
125 
15 
29 
43 
62 
79 
935 
1

568 
109 
120 
14 
18 
24 
36 
48 
60 
71 
82 
94 
100 
111 
1268 
137 
15 
18 
228 
31 
41 
61 
74 
86 
15 
312 
372 
46 
71 
889 
11 
195 
237 
27 
324 
34 
351 
361 
942 
402 
414 
441 
555 
381 
515 
1131 
577 
616 
658 
690 
1367 
693 
725 
746 
792 
106 
121 
15 
18 
195 
23 
28 
37 
506 
481 
56 
72 
89 
1323 
471 
517 
557 
444 
71 
2668 
552 
259 
536 
114 
151 
211 
249 
288 
319 
36 
45 
55 
639 
71 
8 
91 
1009 
119 
142 
187 
24 
29 
40 
515 
60 
1093 
66 
1145 
74 
304 
314 
89 
927 
360 
372 
995 
605 
128 
174 
193 
222 
2533 
430 
844 
897 
264 
119 
147 
191 
30 
65 
760 
907 
1042 
308 
378 
542 
577 
602 
638 
990 
306 
238 
485 
543 
218 
170 
333 
390 
430 
722 
836 
2426 
424 
492 
1897 
2384 
38 
442 
402 
449 
680 
559 
576 
76 
848 
448 
654 
427 
485 
508 
535 
554 
577 
617 
656 
929 
567 
607 
629 
644 
690 
721 
928 
548 
572 
589 
605 
629 
662 
775 
401 
1084 
114 
604 
131 
15 
18 
20 
230 
27 
32 
41 
51 
60 
70 
850 
487 
559 
597 
636 
397 
451 
476 

26 
29 
330 
40 
50 
64 
85 
178-17 
179-35 
180-47 
181-69 
182-37 
184-38 
185-56 
80-11 
80-37 
80-50 
138 
80-79 
167B 
81-16 
24 
81-30 
321 
3491 
3597 
375 
53 
94 
80-34 
81-38 
166-10 
80-15 
80-07 
81-25 
115 
14 
17 
205 
246 
269 
289 
31 
35 
3817 
50 
71 
97 
10 
11 
82-30 
82-37 
18 
230 
262 
30 
352 
40 
44 
4912 
5228 
63 
89 
147-01 
147-68 
150-78 
101 
15 
162-02 
19 
80-19 
80-25 
80-33 
80-39 
29 
35 
3646 
80-72 
3734 
3936 
441 
49 
75 
107 
125 
156 
180 
224 
260 
322 
407 
4715 
52 
64 
77 
95 
82-55 
82-64 
82-70 
82-77 
82-83 
105 
12 
139 
152 
17 
21 
36 
53 
67 
80 
93 
15 
228 
259 
285 
303 
316 
329 
343 
357 
375 
393 
411 
433 
106 
144 
170 
226 
241 
254 
267 
279 
310 
46 
78 
96 
11 
1308 
141 
151 
180 
25 
37 
427 
4690 
57 
7 
90 
103 
122 
15 
18 
21 
236 
261 
33 
3898 
41 
56 
739 
87 
132 
1559 
17 
206 
228 
251 
30 
36 
44 
471 
56 
71 
93 
112 
145 
197 
23 
1 
374 
390 
14 
427 
160 
17 
182 
2 
481 
51 
539 
29 
62 
38 
79 
86 
49 


...

2905 
2945 
3009 
3022 
3036 
3225 
825 
918 
2916 
2984 
3005 
3007 
3211 
3244 
781 
3038 
788 
2927 
3312 
3387 
762 
796 
3235 
3318 
3351 
637 
1126 
1196 
2001 
2025 
2064 
2122 
1154 
1920 
1945 
1969 
2013 
2112 
2160 
1007 
2012 
2027 
2048 
2120 
2160 
2121 
2153 
832 
2019 
2060 
2096 
933 
1919 
1936 
1954 
1980 
938 
1814 
1851 
982 
1016 
1914B 
1931 
1945 
1958 
1080 
1820 
1834 
1846 
1858 
1523 
1560 
1824 
1188 
1241 
1753 
1821 
1862 
1917 
1945 
1124 
1819 
1842 
1860 
1157 
1199 
1561 
1604 
1629 
1650 
1710 
1725 
1825 
1857 
1086 
1554 
1574 
1611 
1626 
1640 
1656 
none
none
none
none
1711 
1625 
1000 
1029 
1562 
1575 
1619 
1634 
1646 
1661 
1719 
1731 
974 
1809 
1825 
1841 
1856 
1875 
847 
870 
1915 
1916 
1936 
1970 
1822 
1858 
690 
1719 
1734 
763 
1651 
1670 
1731 
809 
1612 
1641 
1706 
1729 
864 
1712 
1732 
602 
620 
637 
none
none
none
none
1991 
626 
1464 
1526 
2627 
1521 
2427 
2523 
1532 
1622 
2457 
2513 
2557 
1651 
1725 
2410 
2420 
2429 
244

169-19 
172-11 
174-22 
138-08 
138-53 
139-24 
140-33 
186-17 
218-04 
218-47 
219-26 
220-09 
118-11 
118-26 
118-39 
118-59 
119-24 
119-39 
120-15 
120-36 
216-30 
218-25 
115-79 
115-95 
116-09 
116-21 
116-33 
116-45 
117-34 
216-15 
219-07 
122-20 
122-35 
123-22 
125-16 
186-18 
187-16 
188-20 
190-10 
191-14 
192-16 
137-14 
137-44 
137-65 
177-11 
177-34 
184-11 
131-19 
132-42 
135-21 
178-37 
179-09 
179-24 
134-52 
174-13 
1187-22 
121-07 
121-27 
121-45 
122-19 
122-38 
122-62 
129-17 
179-14 
186-04 
186-23 
188-03 
188-42 
189-20 
119-17 
119-38 
120-18 
120-37 
121-01 
121-17 
122-14 
130-28 
193-42 
195-05 
197-11 
198-17 
117-20 
118-05 
118-19 
118-39 
119-11 
119-21 
120-24 
120-62 
201-46 
205-04 
114-14 
114-37 
114-64 
115-61 
116-11 
116-33 
117-31 
205-12 
205-19 
205-26 
114-07 
114-22 
114-35 
114-48 
114-60 
115-11 
115-31 
115-67 
210-14 
212-32 
114-03 
114-22 
114-37 
114-53 
115-02 
115-31 
115-45 
115-63 
116-17 
116-35 
203-08 
116-12 
116-29 
117-09 

13-02 
13-15 
13-32 
7-17 
7-37 
9-23 
123-01 
124-03 
125-09 
126-18 
3-11 
3-34 
5-15 
6-14 
7-24 
119-25 
119-41 
120-11 
120-40 
121-15 
121-31 
122-07 
3-16 
349 
5-14 
6-23 
7-21 
11-06 
11-43 
129-05 
13-09 
13-21 
13-32 
131-09 
138-07 
9-16 
9-32 
119-10 
121-03 
122-16 1/2 
15-18 
18-12 
18-25 
18-36 
20-08 
20-25 
20-45 
22-11 
22-28 
22-49 
23-29 
23-53 
110-29 
112-19 
114-12 
115-21 1/2 
118-07 
119-19 
12-19 
12-34 
120-07 
14-10 
14-17 
14-23 
14-52 
10-17 
11-07 
114-47 
115-02 
117-11 
12-12 
120-11 
5-12 
5-36 
7-08 
9-19 
9-40 
104-11 
104-30 
104-52 
106-07 
108-19 
108-32 
108-49 
46-04 
49-01 
104-23 
104-41 
104-54 
104-68 
108-18 
108-31 
108-45 
108-58 
109-35 
43-04 
45-18 
101-21 
102-11 
102-29 
102-45 
45-08 
98-43 
101-17 
102-34 
40-10A 
41-29 
96-25 
97-20 
98-18 
99-22 
41-19 
41-86 
43-19 
92-38 
94-10 
94-22 
94-33 
94-62 
95-20 
96-32 
42-05 
42-26 
42-48 
43-22 
92-12 
42-11 
42-84 
43-02 
44-41 
90-23 
40-11 
40-35 
40-45 
40-55A 
41-13 
94-21 
97

8 
3 
33 
10 
113 
4005 
9 
665 
12 
79 
34 
12 
21 
94 
364 
9 
2 
4 
10 
6 
19 
12 
84 
26 
3 
8 
5 
12 
2 
80 
20 
19 
37 
19 
5 
6 
18 
12 
18 
1 
40 
33 
39 
18 
27 
28 
254 
7 
22 
49 
22 
87 
27 
88 
9 
47 
800 
11 
9 
25 
223 
322 
25 
35 
19 
6 
332 
2 
7 
21 
16 
44 
34 
10 
176 
69 
19 
7 
58 
29 
59 
118 
315 
10 
7 
27 
36 
1 
65 
40 
7 
3 
9 
8 
1180 
1362 
1193 
6 
24 
12 
4 
26 
1 
1121 
110 
37 
15 
369 
3 
4 
12 
19 
29 
174 
173 
179 
11 
364 
141 
10 
3 
72 
30 
149 
17 
6 
19 
6 
35 
24 
3 
6 
4 
15 
22 
120 
10 
29 
795 
40 
11 
13 
3 
6 
27 
16 
18 
36 
23 
1 
1 
2 
33 
34 
97 
2 
17 
79 
4 
290 
61 
78 
98 
47 
15 
28 
36 
7 
148 
128 
1850 
16 
16 
2 
19 
15 
29 
38 
15 
18 
10 
44 
43 
95 
203 
112 
1 
10 
28 
7 
9 
11 
6 
6 
11 
47 
9 
52 
37 
20 
4 
56 
9 
15 
3 
8 
18 
74 
12 
35 
71 
34 
14 
79 
9 
3 
68 
4 
31 
2 
23 
10 
23 
9 
8 
5 
23 
32 
15 
1017 
21 
59 
24 
15 
66 
7 
9 
23 
149 
6 
12 
29 
19 
19 
11 
26 
2 
23 
4 
267 
205 
14 
83 
51 
46 
164 
1

33 
1032 
1597 
1550 
197 
56 
305 
588 
454 
12 
755 
414 
39 
20 
78 
57 
70 
837 
626 
797 
34 
14 
678 
117 
669 
798 
11 
790 
737 
677 
212 
283 
812 
817 
53 
7 
31 
150 
120 
85 
68 
20 
1200 
21 
35 
10 
31 
19 
14 
277 
390 
14 
310 
283 
251 
400 
309 
235 
27 
27 
498 
3 
420 
3337 
31 
160 
46 
25 
2615 
100 
44 
15 
1 
2 
1 
22 
19 
90 
145 
250 
243 
290 
316 
210 
2190 
165 
388 
146 
73 
51 
64 
8 
95 
32 
77 
80 
17 
39 
8 
24 
1394 
76 
72 
63 
96 
83 
16 
77 
55 
50 
79 
63 
1215 
22 
1179 
44 
181 
89 
25 
2 
14 
83 
90 
1456 
7 
1121 
1510 
244 
1234 
105 
138 
44 
109 
1541 
1433 
1525 
1466 
1321 
701 
227 
219 
1104 
1473 
1416 
1423 
1528 
1233 
205 
1314 
1435 
49 
71 
1401 
1440 
1134 
1434 
1304 
1124 
1229 
36 
55 
597 
208 
849 
13 
132 
22 
144 
951 
699 
642 
28 
219 
620 
565 
465 
616 
312 
426 
515 
590 
533 
510 
462 
418 
310 
381 
232 
168 
157 
212 
235 
228 
258 
321 
374 
165 
115 
121 
149 
180 
232 
464 
215 
137 
35 
60 
106 
129 
11 
35 
84

In [1]:
# -*- coding: utf-8 -*-

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "sample_ny.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS,
                  problem_chars = PROBLEMCHARS, default_tag_type = 'regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]
        for child in element:
            tag = {}
            if PROBLEMCHARS.search(child.attrib["k"]):
                continue
        
            elif LOWER_COLON.search(child.attrib["k"]):
                tag_type = child.attrib["k"].split(':',1)[0]
                tag_key = child.attrib["k"].split(':',1)[1]
                tag["key"] = tag_key
                if child.attrib["k"] == 'addr:street':
                        tag["value"] = update_name(child.attrib["v"], mapping, regex)
                elif child.attrib['k'] == 'addr:zipcode':
                    tag['value'] = update_zipname(child.attrib['v'])
                elif child.attrib['k'] == 'addr:housenumber':
                    tag['value'] = cleanHouse(child.attrib['v'])
                  
                if tag_type:
                    tag["type"] = tag_type
                else:
                    tag["type"] = 'regular'
                    tag["id"] = element.attrib["id"]
                    tag["value"] = child.attrib["v"]
            else:
                tag["value"] = child.attrib["v"]
                tag["key"] = child.attrib["k"]
                tag["type"] = "regular"
                tag["id"] = element.attrib["id"]
            if tag:
                tags.append(tag)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for child in element:
            nd = {}
            tag = {}
            if child.tag == 'tag':
                if PROBLEMCHARS.search(child.attrib["k"]):
                    continue
                elif LOWER_COLON.search(child.attrib["k"]):
                    tag_type = child.attrib["k"].split(':',1)[0]
                    tag_key = child.attrib["k"].split(':',1)[1]
                    tag["key"] = tag_key
                    if child.attrib["k"] == 'addr:street':
                        tag["value"] = update_name(child.attrib["v"], mapping, regex)
                    elif child.attrib['k'] == 'addr:zipcode':
                        tag['value'] = update_zipname(child.attrib['v'])
                    elif child.attrib['k'] == 'addr:housenumber':
                        tag['value'] = cleanHouse(child.attrib['v'])
                    if tag_type:
                        tag["type"] = tag_type
                    else:
                        tag["type"] = 'regular'
                        tag["id"] = element.attrib["id"]
                        tag["value"] = child.attrib["v"]
    
                else:
                    tag["value"] = child.attrib["v"]
                    tag["key"] = child.attrib["k"]
                    tag["type"] = "regular"
                    tag["id"] = element.attrib["id"]
                if tag:
                    tags.append(tag)
                    
            elif child.tag == 'nd':
                nd['id'] = element.attrib["id"]
                nd['node_id'] = child.attrib["ref"]
                nd['position'] = len(way_nodes)
            
                if nd:
                    way_nodes.append(nd)
            else:
                continue
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

UnicodeEncodeError: 'charmap' codec can't encode characters in position 32-37: character maps to <undefined>

In [ ]:
import sqlite3
import csv
from pprint import pprint

In [ ]:
sqlite_file = 'datawrangle.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(sqlite_file)

In [ ]:
# Get a cursor object
cur = conn.cursor()

In [ ]:
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

In [ ]:
# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
# commit the changes
conn.commit()

In [ ]:
# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes_tags.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'],i['value'], i['type']) for i in dr]